### Carga de Base de datos y Librerias

In [1]:
# Cargamos todas las librerias que vamos a necesitar y abrimos la ruta al repositorio

# PARTE 1
!pip install openpyxl
!pip install matplotlib pandas
!pip install pandas pyreadstat
!pip install pandas numpy scikit-learn matplotlib
!pip install seaborn

import pandas as pd
import os # Exportamos este módulo para trabajar con carpetas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# PARTE 2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score


#Cargamos el archivo desde el directorio del GitHub
base_dir = os.getcwd() # Construye la ruta relativa al archivo CSV desde el directorio actual

In [50]:
excel_path = os.path.join(base_dir, 'usu_hogar_T124.xlsx')
hogar_eph24 = pd.read_excel(excel_path)
hogar_eph04 = pd.read_stata('Hogar_t104.dta')
datos_individual = pd.read_csv('datos_individual.csv', low_memory=False)

### Limpieza de Datos

#### Creamos base única

In [51]:
# NOS QUEDAMOS CON LOS DATOS DEL GRAN BUENOS AIRES en las dos bases de datos

# 2004
hogar_eph04= hogar_eph04[hogar_eph04['region']=='Gran Buenos Aires']
# 2024
hogar_eph24=hogar_eph24[hogar_eph24['REGION']==1]

In [52]:
# COINCIDENCIA DE COLUMNAS

#2004
hogar_eph04.columns = hogar_eph04.columns.str.lower()
#2024
hogar_eph24.columns = hogar_eph24.columns.str.lower()


combined_df = pd.concat([hogar_eph24, hogar_eph04], ignore_index=True, sort=False)

# Identifica columnas no compartidas
columns_df1 = set(hogar_eph04.columns)
columns_df2 = set(hogar_eph24.columns)

only_in_df1 = columns_df1 - columns_df2
only_in_df2 = columns_df2 - columns_df1

print("Columnas en 2004 pero no en 2024:", only_in_df1)
print("Columnas en 2024 pero no en 2004:", only_in_df2)

columns_to_remove_2004 = only_in_df1
columns_to_remove_2024 = only_in_df2

# Elimina las columnas de 2004
hogar_eph04 = hogar_eph04.drop(columns=columns_to_remove_2004, errors='ignore')

# Elimina las columnas de 2024
hogar_eph24 = hogar_eph24.drop(columns=columns_to_remove_2024, errors='ignore')

Columnas en 2004 pero no en 2024: {'idimph'}
Columnas en 2024 pero no en 2004: {'pondih'}


In [53]:
# CREAM0S UNICO DATA FRAME
df_hogar = pd.concat([hogar_eph04, hogar_eph24], ignore_index=True, sort=False)

In [54]:
df_hogar

,codusu,nro_hogar,realizada,ano4,trimestre,region,mas_500,aglomerado,pondera,iv1,...,rdeccfr,gdeccfr,pdeccfr,adeccfr,vii1_1,vii1_2,vii2_1,vii2_2,vii2_3,vii2_4
0,125098,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,Departamento,...,06,06,,04,1.0,0.0,98.0,0.0,0.0,0.0
1,125397,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018.0,Departamento,...,00,00,,00,1.0,0.0,98.0,0.0,0.0,0.0
2,125515,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486.0,Departamento,...,07,07,,05,1.0,0.0,98.0,0.0,0.0,0.0
3,125558,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,Departamento,...,08,08,,06,2.0,0.0,3.0,4.0,0.0,0.0
4,125599,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1471.0,Casa,...,04,04,,02,2.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,TQRMNOPSSHKMROCDEIJAH00861731,1.0,1,2024.0,1,1,S,33,3772.0,1,...,10,10.0,NaN,10,1,0,96,0,0,0
4892,TQRMNOPWVHJLSSCDEIJAH00794914,1.0,1,2024.0,1,1,S,33,1862.0,1,...,2,2.0,NaN,2,4,2,98,0,0,0
4893,TQRMNOQQWHKMOQCDEIJAH00861732,1.0,1,2024.0,1,1,S,33,2639.0,1,...,12,12.0,NaN,12,2,0,98,0,0,0
4894,TQRMNOSXRHKMNMCDEIJAH00861733,1.0,1,2024.0,1,1,S,33,1497.0,2,...,10,10.0,NaN,10,1,0,98,0,0,0


In [55]:
# Crear las columnas dummy para los diferentes componentes
num_columnas = datos_individual.shape[1]
print(f"El DataFrame tiene {num_columnas} columnas.")

datos_individual['componente'] = datos_individual['componente'].astype(str)


df_individual_dummy = pd.get_dummies(datos_individual['componente'], prefix='componente')



for col in df_individual_dummy.columns:    # SIN ITERACION: df_individual_dummy['componente_1'] = df_individual_dummy['componente_1'] + df_individual_dummy['componente_1.0'] Eliminar la columna redundante 'componente_1.0' df_individual_dummy = df_individual_dummy.drop(columns=['componente_1.0'])
    if '.' in col:  # Si la columna tiene un sufijo (como '.0')
        base_col = col.split('.')[0]  # Obtener el nombre base de la columna, sin el sufijo
        if base_col in df_individual_dummy.columns:
            # Sumar las columnas con el mismo nombre base
            df_individual_dummy[base_col] = df_individual_dummy[base_col] + df_individual_dummy[col]
            # Eliminar la columna con el sufijo
            df_individual_dummy = df_individual_dummy.drop(columns=[col])

print(df_individual_dummy)


datos_individual = datos_individual.drop_duplicates(subset=['codusu', 'nro_hogar'])
datos_individual= pd.concat([datos_individual, df_individual_dummy], axis=1)
datos_individual.drop('componente', axis=1, inplace=True)

El DataFrame tiene 172 columnas.
       componente_1  componente_10  componente_11  componente_12  \
0              True          False          False          False   
1              True          False          False          False   
2              True          False          False          False   
3             False          False          False          False   
4             False          False          False          False   
...             ...            ...            ...            ...   
13279         False          False          False          False   
13280         False          False          False          False   
13281         False          False          False          False   
13282         False          False          False          False   
13283         False          False          False          False   

       componente_13  componente_14.0  componente_15.0  componente_2  \
0              False            False            False         False   
1     

In [56]:
# Hacer el merge con el parámetro indicator=True para agregar la columna '_merge'

#df_hogar = df_hogar.drop_duplicates(subset=['codusu', 'nro_hogar'])
print(set(df_hogar.columns) & set(datos_individual.columns))

df_completa = pd.merge(df_hogar, datos_individual, on=['codusu', 'nro_hogar'], indicator=True)
print(df_completa.columns.tolist())

# Mostrar los resultados
print(f"Filas solo en df_hogar: {len(df_hogar)}")
print(f"Filas solo en datos_individual: {len(datos_individual)}")
print(f"Filas en ambos DataFrames: {len(df_completa)}")


print(f"Duplicados en df_hogar: {df_hogar[['codusu', 'nro_hogar']].duplicated().sum()}")
print(f"Duplicados en datos_individual: {datos_individual[['codusu', 'nro_hogar']].duplicated().sum()}")
# Ver las filas que no tienen coincidencia
#print("\nFilas solo en df_hogar (sin coincidencia en datos_individual):")
#print(solo_df_hogar)


# Verifica si hay valores nulos en las claves de unión
print("Valores nulos en 'codusu' de df_hogar:", df_hogar['codusu'].isnull().sum())
print("Valores nulos en 'nro_hogar' de df_hogar:", df_hogar['nro_hogar'].isnull().sum())
print("Valores nulos en 'codusu' de datos_individual:", datos_individual['codusu'].isnull().sum())
print("Valores nulos en 'nro_hogar' de datos_individual:", datos_individual['nro_hogar'].isnull().sum())


coincidencias_codusu = df_hogar['codusu'].isin(datos_individual['codusu']).sum()
coincidencias_nro_hogar = df_hogar['nro_hogar'].isin(datos_individual['nro_hogar']).sum()



print(f"Coincidencias de 'codusu': {coincidencias_codusu} de {len(df_hogar)} en df_hogar están en datos_individual")
print(f"Coincidencias de 'nro_hogar': {coincidencias_nro_hogar} de {len(df_hogar)} en df_hogar están en datos_individual")


#print("\nFilas solo en datos_individual (sin coincidencia en df_hogar):")
#print(datos_individual[['codusu', 'nro_hogar']])


completos = df_hogar[df_hogar['codusu'].isin(datos_individual['codusu']) & df_hogar['nro_hogar'].isin(datos_individual['nro_hogar'])]

print(f"Filas con coincidencia completa (tanto 'codusu' como 'nro_hogar'): {len(completos)}")
# Combinar las columnas que tienen los mismos nombres en ambos DataFrames
columns_to_combine = []

{'ideccfr', 'pdeccfr', 'adeccfr', 'gdeccfr', 'deccfr', 'decifr', 'itf', 'pdecifr', 'pondera', 'nro_hogar', 'aglomerado', 'gdecifr', 'rdecifr', 'codusu', 'ipcf', 'trimestre', 'region', 'rdeccfr', 'mas_500', 'idecifr', 'adecifr', 'ano4'}
['codusu', 'nro_hogar', 'realizada', 'ano4_x', 'trimestre_x', 'region_x', 'mas_500_x', 'aglomerado_x', 'pondera_x', 'iv1', 'iv1_esp', 'iv2', 'iv3', 'iv3_esp', 'iv4', 'iv5', 'iv6', 'iv7', 'iv7_esp', 'iv8', 'iv9', 'iv10', 'iv11', 'iv12_1', 'iv12_2', 'iv12_3', 'ii1', 'ii2', 'ii3', 'ii3_1', 'ii4_1', 'ii4_2', 'ii4_3', 'ii5', 'ii5_1', 'ii6', 'ii6_1', 'ii7', 'ii7_esp', 'ii8', 'ii8_esp', 'ii9', 'v1', 'v2', 'v21', 'v22', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19_a', 'v19_b', 'ix_tot', 'ix_men10', 'ix_mayeq10', 'itf_x', 'decifr_x', 'idecifr_x', 'rdecifr_x', 'gdecifr_x', 'pdecifr_x', 'adecifr_x', 'ipcf_x', 'deccfr_x', 'ideccfr_x', 'rdeccfr_x', 'gdeccfr_x', 'pdeccfr_x', 'adeccfr_x', 'vii1_1', 'vii1_

In [57]:
columns_to_combine = []
for col in df_hogar.columns:
    if col in datos_individual.columns:
        # Verificar si las columnas con sufijos _x y _y existen
        if col + '_x' in df_completa.columns and col + '_y' in df_completa.columns:
            
            # Comprobar si alguna de las columnas es de tipo numérico
            is_numeric_x = pd.api.types.is_numeric_dtype(df_completa[col + '_x'])
            is_numeric_y = pd.api.types.is_numeric_dtype(df_completa[col + '_y'])
            
            if is_numeric_x and is_numeric_y:
                # Si ambas columnas son numéricas, quedarse con la que tiene más datos no NaN
                if df_completa[col + '_x'].notna().sum() >= df_completa[col + '_y'].notna().sum():
                    df_completa[col] = df_completa[col + '_x']
                else:
                    df_completa[col] = df_completa[col + '_y']
            elif is_numeric_x:
                # Si solo la columna _x es numérica
                df_completa[col] = df_completa[col + '_x']
            elif is_numeric_y:
                # Si solo la columna _y es numérica
                df_completa[col] = df_completa[col + '_y']
            else:
                # Si ninguna de las columnas es numérica, se podría usar alguna lógica adicional,
                # pero por ahora se deja a las dos columnas con valores no numéricos.
                # Aquí puedes agregar lo que desees hacer si ambas son no numéricas.
                df_completa[col] = df_completa[col + '_x'].combine_first(df_completa[col + '_y'])
            
            # Eliminar las columnas con sufijos _x y _y
            df_completa.drop([col + '_x', col + '_y'], axis=1, inplace=True)

In [58]:
df_completa
#df_completa.to_csv("C:/Users/aula/Downloads/daaaaaaa.csv", index=False)

,codusu,nro_hogar,realizada,iv1,iv1_esp,iv2,iv3,iv3_esp,iv4,iv5,...,gdecifr,pdecifr,adecifr,ipcf,deccfr,ideccfr,rdeccfr,gdeccfr,pdeccfr,adeccfr
0,125098,1.0,Sí,Departamento,,3.0,Mosaico/baldosa/madera/cerámica/alfombra,,Membrana/cubierta asfáltica,Sí,...,2.0,,1.0,368.0,7.0,,6.0,6.0,,4.0
1,125515,1.0,Sí,Departamento,,2.0,Mosaico/baldosa/madera/cerámica/alfombra,,Membrana/cubierta asfáltica,Sí,...,3.0,,2.0,440.0,7.0,,7.0,7.0,,5.0
2,125558,1.0,Sí,Departamento,,4.0,Mosaico/baldosa/madera/cerámica/alfombra,,Membrana/cubierta asfáltica,Sí,...,9.0,,8.0,532.5,8.0,,8.0,8.0,,6.0
3,125599,1.0,Sí,Casa,,4.0,Mosaico/baldosa/madera/cerámica/alfombra,,Membrana/cubierta asfáltica,Sí,...,7.0,,5.0,208.0,4.0,,4.0,4.0,,2.0
4,125783,1.0,Sí,Departamento,,5.0,Mosaico/baldosa/madera/cerámica/alfombra,,N/S. Depto. de propiedad horizontal,Sí,...,5.0,,3.0,150.0,3.0,,3.0,3.0,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668,TQRMNOPSSHKMROCDEIJAH00861731,1.0,1,1,NaN,5,1,NaN,2,2,...,10.0,NaN,10.0,1500000.0,10.0,NaN,10.0,10.0,NaN,10.0
4669,TQRMNOPWVHJLSSCDEIJAH00794914,1.0,1,1,NaN,3,1,NaN,4,1,...,3.0,NaN,4.0,84750.0,2.0,NaN,2.0,2.0,NaN,2.0
4670,TQRMNOQQWHKMOQCDEIJAH00861732,1.0,1,1,NaN,3,1,NaN,2,1,...,12.0,NaN,12.0,0.0,12.0,NaN,12.0,12.0,NaN,12.0
4671,TQRMNOSXRHKMNMCDEIJAH00861733,1.0,1,2,NaN,4,1,NaN,1,1,...,7.0,NaN,8.0,740000.0,10.0,NaN,10.0,10.0,NaN,10.0


####  Valores faltantes 

In [59]:
# Eliminar las columnas donde todas las filas son NaN
df_completa.replace('...', np.nan, inplace=True)
df_completa.replace(0, np.nan, inplace=True)
df_completa.replace(-9, np.nan, inplace=True)
df_completa.replace(9, np.nan, inplace=True)
# Eliminar las columnas que contienen solo NaN
df_completa = df_completa.dropna(axis=1, how='all')

# SACA 2

C:\Users\Mili\AppData\Local\Temp\ipykernel_6456\2453281794.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_completa.replace(0, np.nan, inplace=True)


In [60]:
# Obtener los nombres de las columnas con valores NaN
print("CON ALGUN NAN:")
col = df_completa.columns[df_completa.isna().any()]
col_list = col.tolist()
print(col_list)

print("SIN NINGUN NAN:")
col_no_nan = df_completa.columns[df_completa.notna().all()]
col_no_nan_list = col_no_nan.tolist()
print(col_no_nan_list)

CON ALGUN NAN:
['iv1_esp', 'iv2', 'iv3', 'iv3_esp', 'iv4', 'iv6', 'iv7', 'iv7_esp', 'iv8', 'iv9', 'iv10', 'iv11', 'iv12_1', 'iv12_2', 'iv12_3', 'ii1', 'ii2', 'ii3', 'ii3_1', 'ii4_1', 'ii4_2', 'ii4_3', 'ii5', 'ii5_1', 'ii6', 'ii6_1', 'ii7', 'ii7_esp', 'ii8', 'ii8_esp', 'ii9', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19_a', 'v19_b', 'ix_tot', 'ix_men10', 'ix_mayeq10', 'vii1_1', 'vii1_2', 'vii2_1', 'vii2_2', 'vii2_3', 'vii2_4', 'h15', 'ch03', 'ch06', 'ch08', 'ch14', 'ch15_cod', 'ch16_cod', 'estado', 'cat_inac', 'pp03c', 'pp03d', 'pp3e_tot', 'pp3f_tot', 'pp03g', 'pp03h', 'pp03i', 'pp03j', 'intensi', 'pp04a', 'pp04b_cod', 'pp04b1', 'pp04b2', 'pp04b3_mes', 'pp04b3_ano', 'pp04b3_dia', 'pp04c', 'pp04c99', 'pp04d_cod', 'pp04g', 'pp05b2_mes', 'pp05b2_ano', 'pp05b2_dia', 'pp05c_1', 'pp05c_2', 'pp05c_3', 'pp05e', 'pp05f', 'pp05h', 'pp06a', 'pp06c', 'pp06d', 'pp06e', 'pp06h', 'pp07a', 'pp07c', 'pp07d', 'pp07e', 'pp07f1', 

In [61]:
#FILAS CON NAN
df_completa = df_completa.dropna(subset=['cat_inac'])
df_completa = df_completa.dropna(subset=['itf'])
#COLUMNAS A ELIMINAR (vARIABLES NO CONSIDERADAS RELEVANTES)
df_completa = df_completa.drop(columns=['iv2', 'iv3', 'iv3_esp', 'iv4', 'iv6', 'iv7', 'iv7_esp', 'iv8', 'iv9', 'iv10', 'iv11']) #caracteristicas de la casa
df_completa = df_completa.drop(columns=['ii3_1', 'ii4_1', 'ii4_2', 'ii4_3', 'ii5', 'ii5_1', 'ii6', 'ii6_1', 'ii7_esp', 'ii8', 'ii8_esp', 'ii9']) #ii3 (ocupa casa para trabajar) y ii7 (regimen de tenencia del hogar) se quedan


In [62]:
df_completa

,codusu,nro_hogar,realizada,iv1,iv1_esp,iv5,iv12_1,iv12_2,iv12_3,ii1,...,gdecifr,pdecifr,adecifr,ipcf,deccfr,ideccfr,rdeccfr,gdeccfr,pdeccfr,adeccfr
0,125098,1.0,Sí,Departamento,,Sí,No,No,No,3.0,...,2.0,,1.0,368.0,7.0,,6.0,6.0,,4.0
1,125515,1.0,Sí,Departamento,,Sí,No,No,No,2.0,...,3.0,,2.0,440.0,7.0,,7.0,7.0,,5.0
5,125854,1.0,Sí,Departamento,,Sí,No,No,No,2.0,...,8.0,,7.0,735.0,NaN,,NaN,NaN,,7.0
11,127045,1.0,Sí,Departamento,,Sí,No,No,No,2.0,...,2.0,,1.0,180.0,4.0,,3.0,4.0,,1.0
13,127406,1.0,Sí,Departamento,,Sí,No,No,No,2.0,...,4.0,,3.0,311.0,6.0,,6.0,6.0,,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4656,TQRMNOSPXHJOONCDEIJAH00861729,1.0,1,1,NaN,1,2,2,2,3.0,...,4.0,NaN,5.0,200000.0,6.0,NaN,5.0,5.0,NaN,6.0
4663,TQRMNOSWWHMNSMCDEIJAH00802091,1.0,1,1,NaN,1,2,2,2,3.0,...,5.0,NaN,5.0,210000.0,6.0,NaN,6.0,6.0,NaN,6.0
4664,TQRMNOSRUHJNSOCDEIJAH00855687,1.0,1,1,NaN,1,2,2,2,3.0,...,4.0,NaN,4.0,170000.0,5.0,NaN,5.0,5.0,NaN,5.0
4666,TQRMNORQPHJOSPCDEIJAH00794445,1.0,1,1,NaN,2,2,2,2,2.0,...,2.0,NaN,2.0,210000.0,6.0,NaN,6.0,6.0,NaN,6.0


In [63]:
column_types = df_completa.dtypes

# Crear una función para identificar columnas con tipos de datos mezclados
def columnas_tipos_mixtos(df):
    columnas_mixtas = []
    for columna in df.columns:
        tipos_unicos = df[columna].apply(type).unique()  # Obtener tipos únicos en la columna
        if len(tipos_unicos) > 1:  # Si hay más de un tipo de dato en la columna
            columnas_mixtas.append(columna)
    return columnas_mixtas

# Llamar a la función para encontrar las columnas con tipos de datos mezclados
columnas_mixtas = columnas_tipos_mixtos(df_completa)

# Imprimir el resultado
print("Columnas con tipos de datos mezclados:", columnas_mixtas)
print("Columnas con tipos de datos mezclados:", len(columnas_mixtas))

Columnas con tipos de datos mezclados: ['realizada', 'iv1', 'iv1_esp', 'iv5', 'iv12_1', 'iv12_2', 'iv12_3', 'ii3', 'ii7', 'v1', 'v2', 'v21', 'v22', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19_a', 'v19_b', 'vii1_1', 'vii1_2', 'vii2_1', 'vii2_2', 'vii2_3', 'ch14', 'ch15_cod', 'ch16_cod', 'pp03c', 'pp03g', 'pp03h', 'pp03i', 'pp03j', 'intensi', 'pp04a', 'pp04b_cod', 'pp04b1', 'pp04c', 'pp04c99', 'pp04d_cod', 'pp04g', 'pp05c_1', 'pp05c_2', 'pp05c_3', 'pp05e', 'pp05f', 'pp05h', 'pp06a', 'pp06e', 'pp06h', 'pp07a', 'pp07c', 'pp07d', 'pp07e', 'pp07f1', 'pp07f2', 'pp07f3', 'pp07f4', 'pp07f5', 'pp07g1', 'pp07g2', 'pp07g3', 'pp07g4', 'pp07g_59', 'pp07h', 'pp07i', 'pp07j', 'pp07k', 'pp09a', 'pp09a_esp', 'pp09c_esp', 'pp10a', 'pp10c', 'pp10d', 'pp10e', 'pp11a', 'pp11b_cod', 'pp11b1', 'pp11c', 'pp11c99', 'pp11d_cod', 'pp11l', 'pp11l1', 'pp11m', 'pp11n', 'pp11o', 'pp11p', 'pp11q', 'pp11r', 'pp11s', 'pp11t', 'idecocur', 'pdecocur', 'idec

####  Variables construidas

In [67]:
df_prueba_v1 = df_completa



# Calcular la cantidad de trabajadores por hogar y codusu (trabajo == 1, o el valor correspondiente)
trabajadores_por_hogar_codusu = df_prueba_v1[df_prueba_v1['estado'] == 3].groupby(['codusu', 'nro_hogar']).size()
print(trabajadores_por_hogar_codusu)
# Calcular el total de personas por hogar y codusu
total_personas_por_hogar_codusu = df_prueba_v1.groupby(['codusu', 'nro_hogar']).size()
print(trabajadores_por_hogar_codusu)
# Calcular la proporción de trabajadores por hogar y codusu
proporcion_trabajadores = trabajadores_por_hogar_codusu / total_personas_por_hogar_codusu

# Unir esta proporción al DataFrame original
df_prueba_v1['proporcion_trabajadores'] = df_prueba_v1.set_index(['codusu', 'nro_hogar']).index.map(proporcion_trabajadores)

# Mostrar el DataFrame con la nueva columna
print(df_prueba_v1[['nro_hogar', 'codusu', 'proporcion_trabajadores_hogar_codusu']])



# 1. Verificar los valores únicos en las columnas clave
print("Valores únicos en la columna 'estado':", df_prueba_v1['estado'].unique())
print("Valores únicos en la columna 'codusu':", df_prueba_v1['codusu'].unique())
print("Valores únicos en la columna 'nro_hogar':", df_prueba_v1['nro_hogar'].unique())

# 2. Verificar si hay valores NaN en las columnas clave
print("Valores NaN en 'estado':", df_prueba_v1['estado'].isna().sum())
print("Valores NaN en 'codusu':", df_prueba_v1['codusu'].isna().sum())
print("Valores NaN en 'nro_hogar':", df_prueba_v1['nro_hogar'].isna().sum())

# 3. Filtrar los datos para solo obtener las filas donde estado es 1 (trabajo)
df_trabajando = df_prueba_v1[df_prueba_v1['estado'] == 1]
print(f"Filas con 'estado' == 1 (trabajando): {df_trabajando.shape[0]} filas")

# 4. Calcular la cantidad de trabajadores por hogar y codusu (trabajo == 1)
trabajadores_por_hogar_codusu = df_trabajando.groupby(['codusu', 'nro_hogar']).size()
print("Cantidad de trabajadores por hogar y codusu:\n", trabajadores_por_hogar_codusu)

# 5. Calcular el total de personas por hogar y codusu
total_personas_por_hogar_codusu = df_prueba_v1.groupby(['codusu', 'nro_hogar']).size()
print("Total de personas por hogar y codusu:\n", total_personas_por_hogar_codusu)

# 6. Calcular la proporción de trabajadores por hogar y codusu
proporcion_trabajadores = trabajadores_por_hogar_codusu / total_personas_por_hogar_codusu
print("Proporción de trabajadores por hogar y codusu:\n", proporcion_trabajadores)

# 7. Unir esta proporción al DataFrame original
df_prueba_v1['proporcion_trabajadores'] = df_prueba_v1.set_index(['codusu', 'nro_hogar']).index.map(proporcion_trabajadores)

# Imprimir los valores únicos en la columna 'estado'
print("Valores únicos en la columna 'estado':", df_prueba_v1['estado'].unique())

codusu                         nro_hogar
125047                         1.0          1
125098                         1.0          1
125511                         1.0          1
125515                         1.0          1
125516                         1.0          1
                                           ..
TQSMNORWVHJMOLCDEIJAH00861465  1.0          1
TQSMNOSPPHJLPLCDEIJAH00802544  1.0          1
TQSMNOSTRHLMKUCDEIJAH00861725  1.0          1
TQSMNOSXRHKMTLCDEIJAH00859944  1.0          1
TQSMNOTQRHMMQTCDEIJAH00802239  1.0          1
Length: 1145, dtype: int64
codusu                         nro_hogar
125047                         1.0          1
125098                         1.0          1
125511                         1.0          1
125515                         1.0          1
125516                         1.0          1
                                           ..
TQSMNORWVHJMOLCDEIJAH00861465  1.0          1
TQSMNOSPPHJLPLCDEIJAH00802544  1.0          1
TQSMNOSTRHLMKUCDE

In [59]:
df_prueba_v1

,codusu,nro_hogar,realizada,iv1,iv1_esp,iv5,iv12_1,iv12_2,iv12_3,ii1,ii2,ii3,ii7,v1,v2,v21,v22,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19_a,v19_b,ix_tot,ix_men10,ix_mayeq10,vii1_1,vii1_2,vii2_1,vii2_2,vii2_3,vii2_4,componente,h15,ch03,ch04,ch06,ch07,ch08,ch09,ch10,ch11,ch12,ch13,ch14,ch15,ch15_cod,ch16,ch16_cod,nivel_ed,estado,cat_ocup,cat_inac,pp02c1,pp02c2,pp02c3,pp02c4,pp02c5,pp02c6,pp02c7,pp02c8,pp02e,pp02h,pp02i,pp03c,pp03d,pp3e_tot,pp3f_tot,pp03g,pp03h,pp03i,pp03j,intensi,pp04a,pp04b_cod,pp04b1,pp04b2,pp04b3_mes,pp04b3_ano,pp04b3_dia,pp04c,pp04c99,pp04d_cod,pp04g,pp05b2_mes,pp05b2_ano,pp05b2_dia,pp05c_1,pp05c_2,pp05c_3,pp05e,pp05f,pp05h,pp06a,pp06c,pp06d,pp06e,pp06h,pp07a,pp07c,pp07d,pp07e,pp07f1,pp07f2,pp07f3,pp07f4,pp07f5,pp07g1,pp07g2,pp07g3,pp07g4,pp07g_59,pp07h,pp07i,pp07j,pp07k,pp08d1,pp08d4,pp08f1,pp08f2,pp08j1,pp08j2,pp08j3,pp09a,pp09a_esp,pp09b,pp09c_esp,pp10a,pp10c,pp10d,pp10e,pp11a,pp11b_cod,pp11b1,pp11b2_mes,pp11b2_ano,pp11b2_dia,pp11c,pp11c99,pp11d_cod,pp11g_ano,pp11g_mes,pp11g_dia,pp11l,pp11l1,pp11m,pp11n,pp11o,pp11p,pp11q,pp11r,pp11s,pp11t,p21,decocur,idecocur,rdecocur,gdecocur,pdecocur,adecocur,tot_p12,p47t,decindr,idecindr,rdecindr,gdecindr,pdecindr,adecindr,v2_m,v3_m,v4_m,v5_m,v8_m,v9_m,v10_m,v11_m,v12_m,v18_m,v21_m,t_vi,ano4,trimestre,region,mas_500,aglomerado,pondera,itf,decifr,idecifr,rdecifr,gdecifr,pdecifr,adecifr,ipcf,deccfr,ideccfr,rdeccfr,gdeccfr,pdeccfr,adeccfr,proporcion_trabajadores_hogar_codusu,proporcion_trabajadores
0,125098,1.0,Sí,Departamento,,Sí,No,No,No,3.0,1.0,No,Otra situación,No,Sí,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1.0,NaN,1.0,1.0,NaN,98.0,NaN,NaN,NaN,1.0,1.0,1.0,1,79.0,4,1.0,Sí,"No asiste, pero asistió",0.0,Secundario,No,03,En esta localidad,,En esta localidad,,3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,368.0,5.0,,4.0,4.0,,3.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,368.0,2004.0,1,1,S,32,1299.0,368.0,2.0,,2.0,2.0,,1.0,368.0,7.0,,6.0,6.0,,4.0,NaN,NaN
1,125515,1.0,Sí,Departamento,,Sí,No,No,No,2.0,1.0,No,Propietario de la vivienda y el terreno,No,Sí,No,No,No,No,No,No,Sí,No,No,No,No,Sí,No,No,No,No,No,No,No,No,1.0,NaN,1.0,1.0,NaN,98.0,NaN,NaN,NaN,1.0,1.0,1.0,2,74.0,4,1.0,Sí,"No asiste, pero asistió",0.0,Primario,Sí,,En esta localidad,,En esta localidad,,2,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,440.0,5.0,,5.0,5.0,,4.0,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN,440.0,2004.0,1,1,S,32,1486.0,440.0,3.0,,3.0,3.0,,2.0,440.0,7.0,,7.0,7.0,,5.0,NaN,NaN
5,125854,1.0,Sí,Departamento,,Sí,No,No,No,2.0,2.0,No,Propietario de la vivienda y el terreno,Sí,Sí,No,No,No,No,No,No,No,Sí,No,No,No,Sí,No,No,No,No,No,No,No,No,2.0,NaN,2.0,1.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0,1.0,2,86.0,5,1.0,Sí,"No asiste, pero asistió",0.0,Secundario,No,02,En esta localidad,,En esta localidad,,3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,690.0,7.0,,7.0,7.0,,5.0,240.0,NaN,NaN,NaN,350.0,NaN,NaN,NaN,100.0,NaN,N

In [54]:
df_prueba_v1

,codusu,nro_hogar,realizada,iv1,iv1_esp,iv5,iv12_1,iv12_2,iv12_3,ii1,ii2,ii3,ii7,v1,v2,v21,v22,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19_a,v19_b,ix_tot,ix_men10,ix_mayeq10,vii1_1,vii1_2,vii2_1,vii2_2,vii2_3,vii2_4,componente,h15,ch03,ch04,ch06,ch07,ch08,ch09,ch10,ch11,ch12,ch13,ch14,ch15,ch15_cod,ch16,ch16_cod,nivel_ed,estado,cat_ocup,cat_inac,pp02c1,pp02c2,pp02c3,pp02c4,pp02c5,pp02c6,pp02c7,pp02c8,pp02e,pp02h,pp02i,pp03c,pp03d,pp3e_tot,pp3f_tot,pp03g,pp03h,pp03i,pp03j,intensi,pp04a,pp04b_cod,pp04b1,pp04b2,pp04b3_mes,pp04b3_ano,pp04b3_dia,pp04c,pp04c99,pp04d_cod,pp04g,pp05b2_mes,pp05b2_ano,pp05b2_dia,pp05c_1,pp05c_2,pp05c_3,pp05e,pp05f,pp05h,pp06a,pp06c,pp06d,pp06e,pp06h,pp07a,pp07c,pp07d,pp07e,pp07f1,pp07f2,pp07f3,pp07f4,pp07f5,pp07g1,pp07g2,pp07g3,pp07g4,pp07g_59,pp07h,pp07i,pp07j,pp07k,pp08d1,pp08d4,pp08f1,pp08f2,pp08j1,pp08j2,pp08j3,pp09a,pp09a_esp,pp09b,pp09c_esp,pp10a,pp10c,pp10d,pp10e,pp11a,pp11b_cod,pp11b1,pp11b2_mes,pp11b2_ano,pp11b2_dia,pp11c,pp11c99,pp11d_cod,pp11g_ano,pp11g_mes,pp11g_dia,pp11l,pp11l1,pp11m,pp11n,pp11o,pp11p,pp11q,pp11r,pp11s,pp11t,p21,decocur,idecocur,rdecocur,gdecocur,pdecocur,adecocur,tot_p12,p47t,decindr,idecindr,rdecindr,gdecindr,pdecindr,adecindr,v2_m,v3_m,v4_m,v5_m,v8_m,v9_m,v10_m,v11_m,v12_m,v18_m,v21_m,t_vi,ano4,trimestre,region,mas_500,aglomerado,pondera,itf,decifr,idecifr,rdecifr,gdecifr,pdecifr,adecifr,ipcf,deccfr,ideccfr,rdeccfr,gdeccfr,pdeccfr,adeccfr
0,125098,1.0,Sí,Departamento,,Sí,No,No,No,3.0,1.0,No,Otra situación,No,Sí,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1.0,NaN,1.0,1.0,NaN,98.0,NaN,NaN,NaN,1.0,1.0,1.0,1,79.0,4,1.0,Sí,"No asiste, pero asistió",0.0,Secundario,No,03,En esta localidad,,En esta localidad,,3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,368.0,5.0,,4.0,4.0,,3.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,368.0,2004.0,1,1,S,32,1299.0,368.0,2.0,,2.0,2.0,,1.0,368.0,7.0,,6.0,6.0,,4.0
1,125515,1.0,Sí,Departamento,,Sí,No,No,No,2.0,1.0,No,Propietario de la vivienda y el terreno,No,Sí,No,No,No,No,No,No,Sí,No,No,No,No,Sí,No,No,No,No,No,No,No,No,1.0,NaN,1.0,1.0,NaN,98.0,NaN,NaN,NaN,1.0,1.0,1.0,2,74.0,4,1.0,Sí,"No asiste, pero asistió",0.0,Primario,Sí,,En esta localidad,,En esta localidad,,2,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,440.0,5.0,,5.0,5.0,,4.0,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN,440.0,2004.0,1,1,S,32,1486.0,440.0,3.0,,3.0,3.0,,2.0,440.0,7.0,,7.0,7.0,,5.0
5,125854,1.0,Sí,Departamento,,Sí,No,No,No,2.0,2.0,No,Propietario de la vivienda y el terreno,Sí,Sí,No,No,No,No,No,No,No,Sí,No,No,No,Sí,No,No,No,No,No,No,No,No,2.0,NaN,2.0,1.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0,1.0,2,86.0,5,1.0,Sí,"No asiste, pero asistió",0.0,Secundario,No,02,En esta localidad,,En esta localidad,,3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No,No,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,NaN,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,,NaN,,0.0,0.0,0.0,0.0,0.0,,0.0,NaN,NaN,NaN,0.0,0.0,,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,,,,,,,NaN,690.0,7.0,,7.0,7.0,,5.0,240.0,NaN,NaN,NaN,350.0,NaN,NaN,NaN,100.0,NaN,NaN,690.0,2004.0,1,1,S,32,1796.0,1470.0,8.0,,8.0,8.0,,7.0,735.0,NaN,,NaN,NaN,,